<a href="https://colab.research.google.com/github/Nautyy9/question_answering/blob/main/Question_Answering_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizer
> We see our tokenizer is taking more arguements than usual, that is because this problem is question answering, we've learned during,( methods for pretrained transfer learning (such that we can have embedding learned before our model)) Transfer_Learning-Steps, so this time our bert model is taking in 2 i/p's
* 1) Question
* 2) Answer (Context + Actual answer)
>
>>In normal document website we see that there's a pretext which display some word of the answers and we get full answer along with whole context when we open the website
>>
 * 1) we  see `truncation = "only-second"` which refer to only pad and truncate the context part not the question
 * 2) we see `max-length= 512` but the problem is our context can be bigger than only 512 words , as we generally see in `docs website`
 * 3) we got `stride = 256` , now from CNN we know stride is the amount of block we skip , in here is the number of words it skip , `but here's a catch` instead of skipping over the context we skip over the Question+Context , i.e we've single block of input let's say our question is of 20 token and context is of 492 token which make 512 in total , so for our stride we skip 256 element of first batch , we still have 20 token for the question and if availble 492 token for context and if not then will be padded with 0's

 ### **So at output we see**
 * Our `input_ids` as the batch of 512 token where we've [CLS=1] at start of Question and [SEP=2] at the end of the Question `same` for every single batch and also we've Context having [CLS=1] and [SEP=2] but the rest token are different unlike question
 * `token_ids_type` : [0,0,0,0,0...... ,1,1...], starting from 0's upto Question length , 1's for the Context --For each batch

 * `offset_mapping` -- now this is important , this has the tuple of each word length , so that we can map our exact index location of the actual answer provided to the corresponding token , this is due to the fact that after tokenizing the index provided by input for actual answer won't match the tokenizer output sentence , because the tokenizer add [CLS] and [SEP] token plus G(cap) behind every token value so the sentence length will always change





In [ ]:
!pip install evaluate
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (DataCollatorWithPadding,create_optimizer,DebertaTokenizerFast)
from transformers import DistilBertTokenizerFast

In [ ]:
CONFIG = {
  "BATCH_SIZE": 1,
  "MAX_LENGTH": 4096,
  "EPOCHS": 3
}

In [ ]:
# dataset = load_dataset("covid_qa_deepset")
dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
answer = 'Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.'

In [ ]:
# answer start at 370 from above we see and the length of it is 95 which give us the answer start and answer end
print(len(answer) + 370)

465


In [ ]:
?DistilBertTokenizerFast

In [ ]:
# model_id = "distilbert-base-uncased"
# tokenizer = DistilBertTokenizerFast.from_pretrained(model_id)

In [ ]:
# Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length. i.e, [152,0.0.0.0..... upto 18 index] hence this form long sequences thef to solve this we've longformer
# To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer.
# Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention.

from transformers import LongformerTokenizerFast,TFLongformerForQuestionAnswering
model_id="allenai/longformer-large-4096-finetuned-triviaqa"
tokenizer = LongformerTokenizerFast.from_pretrained(model_id)

In [ ]:
model_id = "microsoft/deberta-base"
tokenizer = DebertaTokenizerFast.from_pretrained(model_id)

In [ ]:
tokenizer_examples = tokenizer(
    dataset['train'][0]["question"],
    dataset['train'][0]['context'],
    truncation="only_second",
    max_length = CONFIG["MAX_LENGTH"],
    stride = 64,
    return_overflowing_tokens = True,
    return_offsets_mapping=True,
    padding="max_length"
)

In [ ]:
print(tokenizer_examples["overflow_to_sample_mapping"])
# see here we got 18 zeros which signifies that for our first input the number of batches[Question(20)+Context(492)] created are 18
#  now we see all zeros but not [0,1,2,3,4,5....17] which signifies increasing batches i.e, we in total have the 18 batches , but that's not the case bcz we are not converting the different input samples into consecutive batches but converting every single individual input content into multiple batches
#  although every single input is converted into batches but internally(content only) not externally(every single input into seqeunce of batches) i.e,
#  ex --> for 50 inputs the batch of 32  give o/p as [0,1....31] - [31...50] , but here we do for first input's content -->  [0...511]-[511...1021]-... and then for next input content , hence for train[0] input we see ["overflow_to_samples"] as 18 0's for train[0] , for train[1] it would be say 15 1's for train[2] say 10 2's and so on


[0]


In [ ]:
print(tokenizer_examples.keys())

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])


In [ ]:
print(len(tokenizer_examples["overflow_to_sample_mapping"]))

1


In [ ]:
len(tokenizer_examples['input_ids'][0])

4096

In [ ]:
yuhu = tokenizer.tokenize('[CLS] What is the main cause of HIV-1 infection in children? [SEP]')
print(yuhu)

['[CLS]', 'ĠWhat', 'Ġis', 'Ġthe', 'Ġmain', 'Ġcause', 'Ġof', 'ĠHIV', '-', '1', 'Ġinfection', 'Ġin', 'Ġchildren', '?', 'Ġ', '[SEP]']


In [ ]:
tokenizer.encode("What is the main cause of HIV-1 infection in children?")

[1, 2264, 16, 5, 1049, 1303, 9, 7947, 12, 134, 7910, 11, 408, 116, 2]

In [ ]:
for ids in tokenizer_examples["input_ids"]:
  print(ids)
  print("--->" , tokenizer.decode(ids))

[0, 3972, 2661, 222, 5, 9880, 2708, 2346, 2082, 11, 504, 4432, 11, 226, 2126, 10067, 1470, 116, 2, 2, 37848, 37471, 28108, 6, 5, 334, 34, 10, 4019, 2048, 4, 497, 1517, 5, 4326, 6919, 18, 1637, 31346, 16, 10, 9030, 9577, 9, 5, 9880, 2708, 4, 29261, 11, 760, 9, 5, 4326, 6919, 8, 2114, 24, 6, 16, 10, 7621, 9577, 9, 4845, 19, 3701, 62, 33161, 19, 5, 7875, 22, 39043, 1459, 1614, 1464, 13292, 4977, 845, 4130, 7, 5, 4326, 6919, 16, 5, 26429, 2426, 9, 5, 25095, 6924, 4, 29261, 639, 5, 32394, 2426, 16, 5, 7461, 26187, 6, 10, 19035, 317, 9, 9621, 8, 12456, 4, 85, 16, 10, 24633, 9, 5, 11491, 26187, 23, 226, 2126, 10067, 6, 1470, 147, 5, 9880, 2708, 2851, 13735, 352, 1382, 7, 6130, 6552, 625, 3398, 208, 22895, 853, 1827, 11, 504, 4432, 4, 497, 5, 253, 9, 5, 1049, 1305, 36, 463, 11, 10, 2228, 516, 14, 15230, 149, 155, 19638, 8, 5, 2610, 25336, 238, 16, 10, 2007, 6, 2297, 7326, 9577, 9, 2708, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
offset_mapping_ls =  [(0, 4), (4, 7), (7, 11), (11, 16), (16, 22), (22, 25), (25, 29), (29, 30), (30, 31), (31, 41), (41, 44), (44, 53), (53, 54), (0, 0), (0, 8), (8, 10), (10, 18), (18, 23), (23, 27), (27, 30), (30, 33), (33, 34), (34, 38), (38, 39), (39, 43), (43, 54), (54, 59), (59, 66), (66, 67), (67, 69), (69, 70), (70, 75), (75, 88), (88, 91), (91, 95), (95, 96), (96, 97), (97, 98), (98, 99), (99, 104), (104, 107), (107, 110), (110, 111), (111, 113), (113, 115), (115, 116), (116, 118), (118, 119), (119, 120), (120, 123), (123, 124), (124, 127), (127, 128), (128, 130), (130, 131), (131, 132), (132, 140), (140, 141), (141, 143), (143, 144), (144, 146), (146, 148), (148, 151), (151, 152), (152, 153), (153, 154), (154, 156), (156, 159), (159, 160), (160, 161), (161, 163), (163, 165), (165, 168), (168, 169), (169, 174), (174, 176), (176, 177), (177, 179), (179, 180), (180, 183), (183, 188), (188, 189), (189, 194), (194, 195), (195, 197), (197, 200), (200, 201), (201, 203), (203, 205), (205, 207), (207, 209), (209, 210), (210, 215), (215, 217), (217, 219), (219, 225), (225, 228), (228, 229), (229, 234), (234, 236), (236, 238), (238, 242), (242, 246), (246, 247), (247, 254), (254, 256), (256, 258), (258, 263), (263, 264), (264, 270), (270, 272), (272, 274), (274, 280), (280, 281), (281, 288), (288, 289), (289, 293), (293, 294), (294, 296), (296, 297), (297, 299), (299, 300), (300, 302), (302, 303), (303, 304), (304, 306), (306, 307), (307, 309), (309, 311), (311, 312), (312, 319), (319, 320), (320, 321), (321, 324), (324, 325), (325, 328), (328, 330), (330, 332), (332, 333), (333, 340), (340, 341), (341, 343), (343, 344), (344, 346), (346, 347), (347, 348), (348, 356), (356, 357), (357, 362), (362, 368), (368, 369), (369, 376)]
print(len(offset_mapping_ls))
#  this gives the number of token we have to iterate to get to the answer or the starting index of the answer token

152


In [ ]:
tf.zeros(shape=(17,) , dtype=tf.int32)

<tf.Tensor: shape=(17,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>

In [ ]:
# we know that the "overflow_to_sample_mapping" and "offset_mapping" is only useful to extract the start_token and end_token of the actual answer among the all the context , while the "offset_mapping" gives the tuple for the all the tokens the ("overflow_to_sample_mapping") will give the number of batches within the whole single input
# with ("overflow_to_sample_mapping") we can provide the info to our model , that among (18) which of these batches, the actual answer lies i.e, starts and ends

# tokenizer_examples["start_positions"] --> gives the info about the answer starting token((352,...)) index among which of the batches
# tokenizer_examples["end_positions"] --->    gives the info about the answer end token((372,...)) index  among which of the batches

sample_mapping = tokenizer_examples.pop("overflow_to_sample_mapping")
offset_mapping= tokenizer_examples.pop("offset_mapping")

tokenizer_examples["start_positions"] = []#[152, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,]
tokenizer_examples["end_positions"] = []#[172, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# what we see that answer start at first batch and at 152'th token index and answer end at first batch and at 172'th token index

# Note !! the token_index  != the sentence starting index
# We need the token_index


In [ ]:
for key, data in enumerate(dataset["train"]):

  answer_start = data["answers"]["answer_start"][0]
  print(data.keys())
  break

dict_keys(['id', 'title', 'context', 'question', 'answers'])


In [ ]:
len(dataset["train"][0]["answers"]["text"][0])

26

In [ ]:
for i , offsets in enumerate(offset_mapping):
  if len(dataset["train"][0]["answers"]["answer_start"]) == 0:
    tokenizer_examples["start_positions"].append(0)
    tokenizer_examples["end_positions"].append(0)
  else:
    start_char = dataset["train"][0]["answers"]["answer_start"][0]
    end_char = start_char+len(dataset["train"][0]["answers"]["text"][0])

    print(start_char, end_char, len(offset_mapping[0]) , len(offset_mapping[0:]))

    found=0
    start_token_position=0
    end_token_position=0

    for j, offset in enumerate(offsets):
      if offset[0] <= start_char and offset[1]>= start_char and found==0:
        start_token_position=j # j ----> 370
        end_token_position=CONFIG["MAX_LENGTH"] # ---> 512, here 512 represent that we've encountered the end of the first batch and for the next if condition we are assigning its actual value, if found in the first batch else we have end pos =
        # 512
        #
        found=1
        print("hehehe",start_token_position, end_token_position, found)
      if offset[1]>=end_char and found==1:
        end_token_position=j
        print("hahaha",end_token_position)
        break
    # since we want our tokenized array to have the indices == no. of the batches for single input batches , where the indicies beside 0's represent that the start/end token is found in that particular batch at the specified number of token , ex->[0,0,231,0...0] means the token is found at 231 index of the 3rd batch(2nd_index)
    tokenizer_examples["start_positions"].append(start_token_position)
    tokenizer_examples["end_positions"].append(end_token_position)

515 541 4096 1
hehehe 135 4096 1
hahaha 142


In [ ]:
print(tokenizer_examples["start_positions"])
print(tokenizer_examples["end_positions"])

[135]
[142]


In [ ]:
def preprocess_function(dataset):
  questions = [q.lstrip() for q in dataset["question"]]
  paragraphs = [p.lstrip() for p in dataset["context"]]

  tokenized_examples = tokenizer(
      questions,
      paragraphs,
      truncation="only_second",
      max_length=CONFIG["MAX_LENGTH"],
      stride=64,
      return_overflowing_tokens=True, # Gives the overflowing_tokens_mapping
      return_offsets_mapping=True, # Gives the offsets key
      padding="max_length",
  )
  # now we are doing popping to save memory consumption and get the returned value
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples['start_positions'] = []
  tokenized_examples['end_positions'] = []

  for i , offsets in enumerate(offset_mapping):
    sample_index = sample_mapping[i]

    start_char = dataset["answers"][sample_index]["answer_start"][0]
    end_char = start_char+len(dataset["answers"][sample_index]["text"][0])
    found=0
    start_token_position=0
    end_token_position=0

    for j, offset in enumerate(offsets):
      if offset[0] <= start_char and offset[1]>= start_char and found==0:
        start_token_position=j # j ----> 370
        end_token_position=CONFIG["MAX_LENGTH"] # ---> 512, here 512 represent that we've encountered the end of the first batch and for the next if condition we are assigning its actual value, if found in the first batch else we have end pos =
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

  return tokenized_examples

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train "].column_names
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10570
    })
})

In [ ]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=CONFIG["BATCH_SIZE"],
)

In [ ]:
tf_val_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=CONFIG["BATCH_SIZE"],
)

In [ ]:
for i in tf_train_dataset.take(10):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[   0, 2264, 6680, ...,    1,    1,    1]])>, 'attention_mask': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[1, 1, 1, ..., 0, 0, 0]])>, 'start_positions': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([36])>, 'end_positions': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([43])>}
{'input_ids': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[   0, 2264,   76, ...,    1,    1,    1]])>, 'attention_mask': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[1, 1, 1, ..., 0, 0, 0]])>, 'start_positions': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([27])>, 'end_positions': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([28])>}
{'input_ids': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[    0, 14229,     5, ...,     1,     1,     1]])>, 'attention_mask': <tf.Tensor: shape=(1, 4096), dtype=int64, numpy=array([[1, 1, 1, ..., 0, 0, 0]])>, 'start_positions': <tf.Tensor: shape=(1,), dty

In [ ]:
# train_dataset=tf_dataset.take(int(0.9*len(tf_dataset)))
# val_dataset=tf_dataset.skip(int(0.9*len(tf_dataset)))

# Modelling


In [ ]:
from transformers import TFDebertaForQuestionAnswering, LongformerTokenizer, TFLongformerForQuestionAnswering
model = TFLongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

All model checkpoint layers were used when initializing TFLongformerForQuestionAnswering.

All the layers of TFLongformerForQuestionAnswering were initialized from the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.


In [ ]:
len(tokenized_dataset["train"])

87599

In [ ]:
int(len(tokenized_dataset["train"])//CONFIG["BATCH_SIZE"])

87599

In [ ]:
CONFIG["BATCHES_PER_EPOCH"] = int(len(tokenized_dataset["train"])//CONFIG["BATCH_SIZE"])
CONFIG["TRAIN_STEPS"] = int(CONFIG["EPOCHS"]*CONFIG["BATCHES_PER_EPOCH"])

In [ ]:
optimizer, schedule = create_optimizer(init_lr=1e-5,num_warmup_steps=0, num_train_steps=CONFIG["TRAIN_STEPS"])

In [ ]:
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer)

In [ ]:
history = model.fit(tf_train_dataset, validation_data =tf_val_dataset, epochs=CONFIG["EPOCHS"], verbose=1)

Epoch 1/3


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.


ResourceExhaustedError: ignored